In [1]:
import numpy as np
import gym

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

Using TensorFlow backend.


# Q-Learning y Deep Q-Learning.

- ### Pablo Melendez
- ### Hector Magaña

### Aprendizaje por Reforzamiento

Como ya sabemos, el objetivo de un agente es actuar o resolver algun problema mediante la ejecucion de alguna accion de acuerdo al ambiente en el que esta y que el mismo agente puede detectar u "observar". Tomando en cuenta esto, el aprendizaje por reforzamiento le permite al agente "aprender" que accion le conviene realizar de acuerdo al ambiente en el que este se encuentre.

Para poder entender mas a fondo este metodo es importante definir lo siguiente:


- **Estado:**

    El ambiente en el que se encuentra el agente y que al cambiar, puede no ser completamente influenciado por el agente, es decir, que a pesar de la accion que el agente realice el estado puede o no cambiar, y si cambia puede no ser de la manera prevista, es decir es no deterministico.


- **Accion:**

    Una accion es lo que agente puede realizar en su ambiente por ejemplo, si estamos entrenando un agente para jugar Blackjack, las acciones podrian ser: *Partir, Pedir una carta mas, Quedarse con las cartas que tiene y Retirarse*.


- **Recompensa:**

    Cada vez que el agente realiza una accion necesita saber si dicha accion fue buena o mala, por lo cual, la recompensa la podemos ver como este valor, donde al realizar una accion negativa podemos obtener una recompensa negativa o nula y si la accion es positiva podemos obtener una recompensa positiva.


- **Policy (Poliza):**

    Es el conjunto de reglas que va a seguir el agente, en la cual vamos a tener todos los estados del ambiente y la accion que el agente va a realizar cuando se encuentre en cada estado.


El objetivo del aprendizaje por reforzamiento es encontrar la poliza donde para cada estado se tenga la accion que le genere la mayor recompensa posible al agente.

La base de este metodo de aprendizaje se tiene en el ***Proceso de Decisiones de Markov*** que nos va a permitir generar un modelo donde podemos expresar la relacion del agente con el estado, sus acciones y sus recompensas y la ***Ecuacion de Bellman*** que nos ayuda a encontrar la poliza mas adecuada.

- **Proceso de Decisiones de Markov (*MDP, Markov Decision Process*)**:

    Nos permite generar un modelo donde se haga visible la relacion que hay entre el agente, las acciones que este puede tomar, las recompensas de dichas acciones y el estado al que puede llevar dicha accion. Por ejemplo:

![Proceso de Decisiones de Markov](https://miro.medium.com/max/1280/1*Uh11rrUKKsHLLRmmv0ss2w.jpeg)
    En la imagen podemos observar tres diferentes estados para el automovil: *Frio, Caliente y Sobrecalentado*, de igual manera podemos observar las acciones que el automovil puede realizar: *Avanzar Rapido y Avanzar Lento*, los estados a los que estas acciones nos llevan y la probabilidad de que eso ocurra.
    
    
- **Ecuacion de Bellman**


$$
Q^{*}(s,a)=\mathbb{E}_{s'\sim\epsilon}[r+\gamma max_{a'}Q^{*}(s',a')|s,a]
$$

Esta ecuacion nos ayuda a saleccionar la mejor accion *a* para un estado *s*, al escoger la accion que pueda dar una mayor recompensa a futuro.

Para esto hay que tomar en cuenta que si una accion en el estado actual nos genera una recompensa, esta recompensa puede disminuir o no ser tan buena a futuro, ahi se explica el valor de $\gamma$ dentro de la ecuacion, ya que esta indica que la recompensa de ejecutar en el estado *s* la accion *a* va a ser igual a la recompensa que dicha accion *a* genere ($r$) mas la recompensa con descuento de ejecutar la mejor accion *a'* desde el estado *s'* ($\gamma max_{a'}Q^{*}(s',a')$) donde $\gamma$ va a ser un factor que reduzca la recompensa que nos de la la mejor accion *a'* para el estado *s'*.

#### Algoritmo Q-Learning

En el aprendizaje por reforzamiento nos vamos a encontrar con dos "tipos": ***Online*** y ***Offline***, cuya diferencia radica en que para el tipo online, el agente conoce la recompensa que le va a generar alguna accion y esta recompensa es negativa, el agente no explora o no va por dicho camino en busca de una poliza, mientras que en el offline el agente tiene que explorar dicha accion para aprender que es mala o que la recompensa que le genera no es buena mientras busca la solucion.

<!-- Empieza la descripcion de que es el algoritmo de Q-Learning -->


El algoritmo de Q-Learning toma el MDP y la ecuacion de Bellman para realizar de manera iterativa la busqueda de la mejor poliza para resolver el problema planteado, de tal manera que para la primer iteracion de cada estado *s* podemos plantear lo siguiente: ¿Cual es la recompensa de aplicar la acción *a* si parto desde este estado *s*?, para resolver dicha pregunta tenemos que ir al estado *s'* derivado de aplicar la accion *a* en el estado *s* y repetir dicho planteamiento pero ahora partiendo del estado *s'* y aplicando una accion *a'*.

Junto con esta modo iterativo de atacar el problema debemos agregarle la "memoria" al agente y debemos darle un peso a dicha memoria ya que de no hacerlo podriamos ciclarnos en una serie de estados que no nos lleven a un estado terminal del problema para esto utilizamos varios parametros agregados a la ecuacion de Bellman:

- ***Gamma ($\gamma$)***: Descuento de la recompensa, este valor nos permite alterar el valor de la recompensa obtenida ya que, como ya se menciono antes, cierta accion en cierto estado puede ser bastante buena al inicio de la solucion pero no tanto al final de la misma, teniendo en cuenta que de llegar a un estado que ya hemos procesado antes seria ir en circulos, por lo que este parametro igual ayuda a salir de dicho ciclo.

- ***Epsilon***: Le da "libertad" al agente, permite que en la busqueda el agente decida realizar o explorar una accion "no tan buena" o con menor recompensa aproximada para abarcar una mayor area en el espacio de busqueda. Hay que recordar que estamos buscando un punto maximo en las recompensas por lo que podriamos llegar a tener optimos locales donde el agente puede atorarse o tomar como la mejor solucion, por lo que el permitirle salir de ahi y buscar en otro lado ayuda a encontrar otra opcion que puede ser mejor.

- ***Delta Epsilon***: Va restringiendo la libertad que tiene el agente de explorar otras soluciones haciendo que lo aprendido tenga mas peso que una accion aleatoria, esto es debido a que aunque es bueno explorar diferentes acciones al inicio, para poder llegar a una solucion debemos continuar y tener en cuenta lo aprendido para poder obtener un valor de recompensa acertado, en vez de estar tomando acciones al azar todo el tiempo y no llegar a resolver el problema.


Con todos estos parametros aplicados a la ecuacion de Bellman y corriendo el algoritmo iterativo, podemos generar una matriz con las dimensiones (estados_posibles x cantidad_acciones) donde  el valor en M\[s,a\] es la recompensa esperada de aplicar la accion *a* en el estado *s*. De ahi queda obtener la mejor recompensa y asignar la accion que genera dicha recompensa como la accion a realizar en dicho estado para nuestra poliza final.

<!-- Termina la descripcion de Q-Learning -->

#### Algoritmo Deep Q-Learning


<!-- Empieza la descripcion de que es el algoritmo de Deep Q-Learning -->


El algoritmo de Deep Q-Learning reemplaza la tabulacion o matriz generada con una aproximacion de la funcion *Q* mediante el uso de una red neuronal que nos permite filtrar o pulir mas la informacion de las entradas en cada una de las capas de la misma red.

<!-- donde la ecuacion de Bellman nos lleva a una serie de funciones de perdida que se buscan minimizar y que cambian con cada iteracion del algoritmo. -->

Un ejemplo de este algoritmo lo encontramos en el articulo [Human-level control through deep reinforcement
learning](https://web.stanford.edu/class/psych209/Readings/MnihEtAlHassibis15NatureControlDeepRL.pdf)

Donde se pasa de la ecuacion de Bellman a un par de funciones de perdida:

$$
L_{i}(\theta_{i})=\mathbb{E}_{s,a,r}[(E_{s'}[yD_{s,a}]-Q(s,a;\theta_{i}))^2]
$$



$$
L_{i}(\theta_{i})=\mathbb{E}_{s,a,r,s'}[(y-Q(s,a;\theta_{i}))^{2}]+E_{s,a,r}[V_{s'}[y]]
$$

Estas funciones ayudan a actualizar los pesos usados en la red neuronal en el momento del entrenamiento y como lo explican los autores el ajuste que estas generan junto con el uso de una memoria y una red neuronal con dos capas ocultas mas la capa de entrada y la capa de salida permiten tratar la informacion de manera que el proceso de aprendizaje sea rapido y sin generar la matriz de relacion entre los estados posibles y las acciones.

Esto es util cuando nos enfrentamos a problemas de grandes dimensiones donde el espacio o cantidad de estados posibles y la cantidad de acciones son bastante grandes generalmente cuando por ejemplo, nuestro estado no es solamente un tablero donde se nos indica que celdas estan ocupadas y que celdas no, sino cuando aparte de tener en cuenta esto debemos considerar factores como tiempo o "vida restante" del agente, en general, resulta util cuando la relacion entre el estado y las acciones no se puede ver de manera lineal o  es dificil representarlo de esta manera.

<!-- Termina la descripcion de Deep Q-Learning -->

#### Diferencias de Deep Q-Learning y Q-Learning

La principal diferencia que encontramos entre estos dos algoritmos, mas alla de las funciones que utilizan para poder determinar en Q-Learning la recompensa de una accion y en Deep Q-Learning las funciones para actualizar los pesos en la red neuronal, es el tamaño del espacio (estado, accion) que pueden manejar, ya que si este espacio es bastante grande al momento de utilizar Q-Learning necesitamos una matriz de dimensiones *(estados_posibles x acciones_posibles)* que nos permita almacenar las diferentes recompensas para cada par (estado, accion), mientras que con Deep Q-Learning la red Neuronal la podemos auxiliar con una memoria de tamaño fijo y que al tener varias capas ocultas nos permite procesar mejor la informacion ya que no solo depende de un factor *gamma* sino que el resultado es influenciado por la salida de todos los elementos de la red que poco a poco iran ajustando sus pesos para llegar al resultado esperado.

#### Dfinimos el algoritmo de Q-Learning.

In [2]:
def sel_accion(Q, s, eps=0.1):
    if np.random.uniform(0,1) < eps:
        return np.random.randint(Q.shape[1])
    else:
        return np.argmax(Q[s])

def QL(env, n_iter=10000, gamma=0.95, alfa=0.01, eps=0.3, d_eps=0.00005):
    
    n_a = env.action_space.n
    n_s = env.observation_space.n

    Q = np.zeros((n_s, n_a))
    
    for it in range(n_iter):
        done = False        
        state = env.reset()
        
        if eps > 0.01:
            eps -= d_eps
        
        while not done:
            action = sel_accion(Q, state, eps)
            next_state, rew, done, info = env.step(action)

            Q[state,action] = Q[state,action] + alfa*(rew + gamma*np.max(Q[next_state]) - Q[state,action])
            state = next_state
            
    return Q

#### Definimos el algoritmo de Deep Q-Learning.

In [3]:
def DeepQL(env, mem_limit = 1000, batch = 50, warmup = 2000, steps = 50000):

    # Semilla para valores aleatorios
    np.random.seed(123)
    env.seed(123)
    
    # Obtenemos la cantidad de acciones
    nb_actions = env.action_space.n

    # Primera capa
    model = Sequential()
    model.add(Dense(nb_actions, input_shape=(1,)+env.observation_space.shape))
              
    # Capas siguientes
    model.add(Dense(16))
    model.add(Dense(16))
    model.add(Dense(16))
    model.add(Dense(nb_actions))
              
    # Estructura de la red
    print(model.summary())

    # Creamos la poliza inicial y compilamos el agente
    memory = SequentialMemory(limit = mem_limit, window_length = 1)
              
    policy = BoltzmannQPolicy()
              
    dqn = DQNAgent(model=model, nb_actions = nb_actions, memory = memory, nb_steps_warmup = warmup,
                   target_model_update = 1e-2, policy = policy)
              
    dqn.compile(Adam(lr = 1e-3), metrics = ['mae'])

    # Entrenamos al agente
    dqn.fit(env, nb_steps = steps, visualize = False, verbose = 0)

    # Testeamos el resultado
    dqn.test(env, nb_episodes = 50, visualize = False)
    
    # Regresamos el agente entrenado
    return dqn

#### Frozen Lake

El problema nos pone al agente sobre un lago congelado donde tiene un punto de partida y un punto objetivo al cual llegar, sin embargo la probabilidad de que el agente se mueva de acuerdo a lo que le indiquemos es baja (0.33) y las recompensas tambien por lo que el entrenar al egente es una tarea complicada ya que de acuerdo a la documentacion esto se debe a los "resbalozo" que es el hielo y a que el agente unicamente recibe una recompensa al llegar al objetivo. [Ver Docs.](http://gym.openai.com/envs/FrozenLake8x8-v0/)

In [4]:
test_env = gym.make('FrozenLake-v0')

test_env.reset()

print(test_env.observation_space)
print(test_env.action_space)


Discrete(16)
Discrete(4)


In [69]:
test_env.reset()
DQr = DeepQL(test_env, mem_limit = 1000, batch = 10, warmup = 10000, steps = 5000)

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_26 (Dense)             (None, 4)                 8         
_________________________________________________________________
dense_27 (Dense)             (None, 16)                80        
_________________________________________________________________
dense_28 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_29 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_30 (Dense)             (None, 4)                 68        
Total params: 700
Trainable params: 700
Non-trainable params: 0
_________________________________________________________________
None
Testing for 50 episodes ...
Episode 1: reward: 0.000, steps: 6
Episode 2: reward: 0.000, steps: 5
Episode 3: reward: 

In [12]:
test_env.reset()
Qr = QL(test_env, n_iter = 1000, gamma = 0.95, alfa = .1, eps = 0.5, d_eps = 0.001)
print(Qr)
    

[[1.18630311e-01 1.18979672e-01 1.31370423e-01 1.18660470e-01]
 [1.39348559e-02 2.10387239e-02 2.04852622e-02 1.27388384e-01]
 [1.32858494e-01 3.89747531e-02 6.07853606e-03 1.39450314e-02]
 [7.30197453e-06 0.00000000e+00 0.00000000e+00 2.10252726e-02]
 [1.60522967e-01 1.29935327e-01 1.05796188e-01 5.17762098e-02]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [1.88762065e-02 6.22666909e-02 1.46565200e-01 1.88092752e-03]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [2.80830987e-02 1.31987004e-01 1.01341796e-01 2.47658546e-01]
 [8.69250533e-02 3.72808709e-01 1.63852343e-01 6.29972954e-02]
 [4.84533777e-01 1.64119935e-01 1.33039663e-01 6.50687479e-02]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [7.34408048e-02 1.92051637e-01 4.45528023e-01 1.08268258e-01]
 [1.14792926e-01 7.16593992e-01 4.91755211e-01 2.25493105e-01]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.000000

In [13]:
obs = test_env.reset()
done = False
while not done:
    a = np.argmax(Qr[obs])
    obs, r, done, info = test_env.step(a)
    test_env.render()  

  (Right)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG


#### Roulette

Este ambiente busca enseñar a jugar a la ruleta de los casinos donde el agente apuesta por un numero y recibe una recompensa positiva si el numero que resulta en la ruleta es diferente de cero y la paridad de dicho numero es igual a la paridad del numero por el que aposto el agente y de igual manera el agente tiene la opcion de retirarse del juego y terminar el ambiente.

En este caso el parametro gamma es casi 1 debido a que la decision que tome el agente en un tiro de la ruleta, no influye en el resultado de tiros posteriores por lo que el descuento de la recompensa de un siguiente intento es casi nulo.

De igual manera alfa que ayuda a determinar el aumento o disminucion de la recompensa para una accion es muy bajo ya que los resultados anteriores.

Epsilon lo establecimos a la mitad para que tenga la misma probabilidad de escoger una accion aleatoria o continuar con la accion (apuesta) que ha estado haciendo (es decir, siga con lo aprendido) y finalmente delta epsilon lo pusimos a un numero muy bajo para que esa probabilidad de que el agente escoja algun numero aleatorio o continue con la apuesta que ha estado haciendo se mantenga lo mas igual posible sin volverla cero porque eso significaria no tomar en cuenta o no darle cada vez mas peso a lo aprendido.

Otra razon de el valor de epsilon y delta epsilon es que al asignarles valores mas grandes, el agente va a aprender que es mejor tomar el camino de retirarse sin perder que el apostar por algun numero.

In [7]:
test_env_2 = gym.make('Roulette-v0')

print(test_env_2.observation_space)
print(test_env_2.action_space)

Discrete(1)
Discrete(38)


In [65]:
test_env_2.reset()

Qri = QL(test_env_2, n_iter=10000, gamma=.9, alfa=.1, eps=0.5, d_eps=0.000001)
print(Qri)

[[-6.78457281e-01 -1.86200459e-01 -2.05500698e-01 -3.77238749e-02
  -1.75521658e-01 -3.29581881e-02 -1.49804470e-01 -2.50040380e-01
  -1.25758798e-01 -1.19733532e-02 -2.04155468e-01 -2.06866312e-03
  -3.78927195e-02 -1.73921741e-01 -5.37172381e-02 -1.69503908e-01
  -6.92683833e-02 -2.20369729e-01 -2.23411254e-01 -2.66097531e-02
  -1.15099913e-01 -5.58173073e-02 -1.09533165e-01 -1.16855642e-01
  -1.94613963e-01 -1.16735719e-01 -3.30731918e-02 -1.63721081e-01
  -1.08644711e-02 -3.83112401e-02 -1.71395985e-01 -3.23493715e-01
  -4.68043455e-02 -8.95599103e-02 -1.62216612e-01 -1.63071902e-01
  -1.50724762e-01  7.92373158e-28]]


In [64]:
obs = test_env_2.reset()
done = False
final_r = 0
a = -10000000000
ind =0
it =0
for i in Qri[0]:
    if i > a:
        ind = it
        a = i
    it = it + 1
while not done:
    obs, r, done, info = test_env_2.step(ind)
    final_r = final_r + r
    print("guess: {}, reward: {}".format(ind, r))
print(final_r)

guess: 25, reward: -1.0
guess: 25, reward: 1.0
guess: 25, reward: -1.0
guess: 25, reward: -1.0
guess: 25, reward: -1.0
guess: 25, reward: 1.0
guess: 25, reward: 1.0
guess: 25, reward: -1.0
guess: 25, reward: -1.0
guess: 25, reward: 1.0
guess: 25, reward: -1.0
guess: 25, reward: -1.0
guess: 25, reward: 1.0
guess: 25, reward: 1.0
guess: 25, reward: 1.0
guess: 25, reward: 1.0
guess: 25, reward: 1.0
guess: 25, reward: 1.0
guess: 25, reward: 1.0
guess: 25, reward: 1.0
guess: 25, reward: -1.0
guess: 25, reward: 1.0
guess: 25, reward: -1.0
guess: 25, reward: 1.0
guess: 25, reward: 1.0
guess: 25, reward: 1.0
guess: 25, reward: -1.0
guess: 25, reward: -1.0
guess: 25, reward: 1.0
guess: 25, reward: 1.0
guess: 25, reward: 1.0
guess: 25, reward: -1.0
guess: 25, reward: -1.0
guess: 25, reward: -1.0
guess: 25, reward: -1.0
guess: 25, reward: 1.0
guess: 25, reward: -1.0
guess: 25, reward: -1.0
guess: 25, reward: -1.0
guess: 25, reward: -1.0
guess: 25, reward: 1.0
guess: 25, reward: 1.0
guess: 25, rew

In [9]:
test_env_2.reset()
DQr = DeepQL(test_env_2, mem_limit = 1000, batch = 10, warmup = 10, steps = 10)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 38)                76        
_________________________________________________________________
dense_7 (Dense)              (None, 16)                624       
_________________________________________________________________
dense_8 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_9 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_10 (Dense)             (None, 38)                646       
Total params: 1,890
Trainable params: 1,890
Non-trainable params: 0
_________________________________________________________________
None
Testing for 50 episodes ...
Episode 1: reward: 11.000, steps: 100
Episode 2: reward: -26.000, steps: 100
Episode 